In [1]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json
from math import isnan
import pandas as pd
from plotly_resampler import FigureResampler
import os
from datetime import datetime
import plotly.express as px
from tqdm import tqdm
import pickle
from copy import deepcopy

def every_nth(nums, nth):
    # Use list slicing to return elements starting from the (nth-1) index, with a step of 'nth'.
    return nums[nth - 1::nth]

def filter_by_time_key(df, key_name, min_time, max_time):
    return df[(df[key_name] > min_time) & (df[key_name] < max_time)]

In [ ]:

well_name = '3s-617_Run2_bha'
time_stamp_str = '20240606161450'
output_base_dir = r'C:\NotOneDrive\Data\merged_input_output_channels'
engine_output_channels_path = os.path.join(output_base_dir, f'{well_name}_WellBalance' + (f'_{time_stamp_str}' if time_stamp_str else '') + '.csv')

sample_rate = 20

packoff_time_key = ''
if '617_Run3' in engine_output_channels_path:
    packoff_time_key = '2023-11-18T12:00:00Z'
    time_key_display_max_limit = '2023-11-18T16:00:00Z'
    time_key_display_min_limit = '2023-11-11T12:00:00Z'
elif '624_Run3' in engine_output_channels_path:
    packoff_time_key = '2024-01-02T12:00:00Z'
    time_key_display_min_limit = '2023-12-25T12:00:00Z'
    time_key_display_max_limit = '2024-01-03T12:00:00Z'
elif '624_Run1' in engine_output_channels_path:
    packoff_time_key = '2023.append('')-12-13T12:00:00Z'
    time_key_display_min_limit = '2023-12-12T12:00:00Z'
    time_key_display_max_limit = '2023-12-18T12:00:00Z'

outputs = pd.read_csv(engine_output_channels_path)
# outputs = filter_by_time_key(outputs, 'time key',time_key_display_min_limit, time_key_display_max_limit)
outputs[['ecd at bit']] = outputs[['ecd at bit']].apply(pd.to_numeric)
outputs[['simulated spp']] = outputs[['simulated spp']].apply(pd.to_numeric)
outputs[['drill string pressure loss']] = outputs[['drill string pressure loss']].apply(pd.to_numeric)
outputs[['m_spp']] = outputs[['m spp']].apply(pd.to_numeric)
print(outputs.columns)

In [ ]:


'''
sspp, m_spp, drill_string_pressure_loss, bit depth, flowrate
'''

def filter_by_rig_state(df: pd.DataFrame, rig_states: list):

    # return df.query('RIG_STATE in rig_states')
    return df[df['RIG_STATE'].isin(rig_states)]

sub_spp_data = outputs[[
    'simulated spp',
    'time key',
    'bit depth'
]].dropna()
m_spp = outputs[['m spp', 'time key']].dropna()

pressure_loss_max = max(
    sub_spp_data['simulated spp'].max(),
    m_spp['m spp'].max()
)
pressure_loss_min = min(
    sub_spp_data['simulated spp'].min(),
    m_spp['m spp'].min()
)

drill_spp = filter_by_rig_state(outputs[['m spp', 'time key', 'RIG_STATE']], [0,1])
trip_in_spp = filter_by_rig_state(outputs[['m spp', 'time key', 'RIG_STATE']], [6])
trip_out_spp = filter_by_rig_state(outputs[['m spp', 'time key', 'RIG_STATE']], [10])

flowrate = outputs[['time key', 'flowrate']].dropna()

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=sub_spp_data['time key'], 
        y=sub_spp_data['simulated spp'], 
        name="Simulated SPP"),
)

fig.add_trace(
    go.Scatter(
        x=sub_spp_data['time key'], 
        y=sub_spp_data['bit depth'], 
        name="bit depth",
        yaxis='y2'),
)

fig.add_trace(
    go.Scatter(
        x=flowrate['time key'], 
        y=flowrate['flowrate'], 
        name="flowrate",
        yaxis='y3'),
)

fig.add_trace(
    go.Scatter(
        x=m_spp['time key'], 
        y=m_spp['m spp'], 
        name="measured spp",
        yaxis='y4'),
)

fig.add_trace(
    go.Scatter(
        x=drill_spp['time key'], 
        y=drill_spp['m spp'], 
        name="drilling measured spp",
        yaxis='y5'),
)

fig.add_trace(
    go.Scatter(
        x=trip_out_spp['time key'], 
        y=trip_out_spp['m spp'], 
        name="tripping out measured spp",
        mode = 'lines+markers',
        yaxis='y6'),
)

fig.update_layout(
    title = fr'{well_name} - Static CHE - Standpipe pressure',
    yaxis=dict(title="Simulated spp (pa)", anchor="free",
        autoshift = True,
        range = [pressure_loss_min, pressure_loss_max]
        ),
    yaxis2=dict(title="bit depth (m)", anchor="free",
        overlaying="y",
        side="right",
        autoshift = True
        ),
    yaxis3=dict(title="flowrate", anchor="free",
        overlaying="y",
        side="right",
        autoshift = True
        ),
    yaxis4=dict(title="measured standpipe pressure (Pa)", anchor="free",
        overlaying="y",
        autoshift = True,
        range = [pressure_loss_min, pressure_loss_max]
        ),
    yaxis5=dict(title="drillig measured standpipe pressure (Pa)", anchor="free",
        overlaying="y",
        autoshift = True,
        range = [pressure_loss_min, pressure_loss_max]
        ),
    yaxis6=dict(title="tripping out measured standpipe pressure (Pa)", anchor="free",
        overlaying="y",
        autoshift = True,
        # range = [pressure_loss_min, pressure_loss_max]
        )
    )

if packoff_time_key:
    fig.add_vline(x=packoff_time_key, line_width=3, line_dash="dash", line_color="green", name = 'Stuckpipe detected')
fig.write_html(fr"images\{well_name}_WellBalance_SPP_{time_stamp_str}.html")
fig.show()

In [ ]:

sub_data_ecd = outputs[[
    'ecd at bit',
    'time key',
    'bit depth'
]].dropna(how='any')

m_ecd = outputs[['time key', 'm ecd at bit']].dropna()
m_ecd = m_ecd[m_ecd['time key'] < sub_data_ecd['time key'].values[-1]]
m_ecd = m_ecd[m_ecd['time key'] > sub_data_ecd['time key'].values[0]]

ecd_max = max(
    m_ecd['m ecd at bit'].max(),
    sub_data_ecd['ecd at bit'].max(),
)
ecd_min = min(
    m_ecd['m ecd at bit'].min(),
    sub_data_ecd['ecd at bit'].min(),
)
'''
measured ecd at bit, simulated ecd at bit
'''

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=sub_data_ecd['time key'], 
        y=sub_data_ecd['ecd at bit'], 
        name='simulated ecd at bit'
    ),
) 

fig.add_trace(
    go.Scatter(
        # mode = 'markers',
        # marker = dict(
        #     size = 2
        # ),
        x=m_ecd['time key'], 
        y=m_ecd['m ecd at bit'], 
        name="measured ecd at bit",
        yaxis='y2'),
)

# fig.add_trace(
#     go.Scatter(
#         # mode = 'markers',
#         # marker = dict(
#         #     size = 5
#         # ),
#         x=result['time_key'], 
#         y=result['ecd_difference'], 
#         name="ecd difference",
#         yaxis='y3'),
# )

fig.add_trace(
    go.Scatter(
        x=sub_data_ecd['time key'], 
        y=sub_data_ecd['bit depth'], 
        name="bit depth",
        yaxis='y3'),
)

# fig.add_trace(
#     go.Scatter(
#         x=sub_data_ecd['time key'], 
#         y=sub_data_ecd['ecd at bit no cuttings'], 
#         name="ecd at bit no cuttings",
#         yaxis='y4'),
# )

# print(len(channel_m_ecd))
# print(channels_output)
# print(len(result))

fig.update_layout(
    title = fr'{well_name} - Static CHE - ECD at bit',
    yaxis=dict(title="simulated ecd at bit",
               range = [ecd_min, ecd_max]),
    yaxis2=dict(title="measured ecd at bit ",        
        anchor="free",
        overlaying="y",
        side="left",
        autoshift = True,
        range = [ecd_min, ecd_max]),
    # yaxis3=dict(title="ecd difference",        
    #     anchor="free",
    #     overlaying="y",
    #     side="left",
    #     autoshift = True),
    yaxis3=dict(title="bit depth",        
        anchor="free",
        overlaying="y",
        side="left",
        autoshift = True),
    # yaxis4=dict(title="ecd at bit no cuttings",        
    #     anchor="free",
    #     overlaying="y",
    #     side="left",
    #     autoshift = True,
    #     range = [ecd_min, ecd_max]),
    )

if (packoff_time_key):
    fig.add_vline(x=packoff_time_key, line_width=3, line_dash="dash", line_color="green", name = 'Stuckpipe detected')
fig.write_html(fr"images\{well_name}_WellBalance_ECD_{time_stamp_str}.html")
fig.show()